# JPPT-0 : J'ai Pas PéTé mon budget calcul!

## Génération simple de textes en français avec des modèles de Markov

par Claude COULOMBE - Ph. D. / consultant en IA appliquée / Lingua Technologies inc.

JPPT-0, un petit générateur de textes en français basé sur un modèle de langue (i.e. un modèle de prédiction du prochain mot) à base de ngrammes et d'une chaîne de Markov. 

Pour cela, il crée un index de trigrammes et de leur fréquence par un dépouillement très rapide d'un petit corpus en quelques secondes sur un micro-ordinateur ordinaire.

Sur l'idée de la chaîne de Markov, un modèle prédictif ainsi constitué peut générer un texte en se basant sur le mot ou gramme qui est le successeur le plus probable de deux mots donnés. Pour démarrer on fournira les deux premiers mots (un germe) ou autrement le générateur fera un choix aléatoire. 

## Pourquoi?

Avec JPPT-0, il est possible de créer des modèles capables de générer des textes assez bluffants, du moins amusants ou poétiques, sans avoir à entraîner des modèles d'apprentissage profond pendant des jours sur des corpus représentant une fraction importante du contenu de la Toile.

GPT-2 pour Generative Pre-training Transformer) auraient été entraînés sur un corpus énorme (40 gigaoctets de textes) pendant une semaine sur une architecture comportant 32 processeurs spécialisés dans les calculs matriciels (TPU: Tensor Processor Unit). On estime les coûts de calcul du modèle comportant 1.5 milliards de paramètres à environ 60 000 dollars canadiens. Ce n’est que la pointe de l’iceberg, car on néglige tous les calculs ayant menés à sa mise au point. On estime que l'entraînement de GPT-3, le successeur de GPT-2, qui comporte 175 milliards de paramètres, aurait coûté plus de 5 millions de dollars canadiens en temps de calcul.

En entraînant à faible coût JPPT-0 sur de petits corpus spécialisés, par exemples des lettres de motivation, il est relativement facile de créer de petites applications sans péter son budget calcul.   


## À défaut d'intelligence artificielle, nous avons créé de l'intelligence superficielle!

Les générateurs de texte auto-attentifs du style GPT-X (j'essaie d'éviter le mot « transformer » qui se rapporte à un dessin animé japonais et qui ne veut strictement rien dire) demeurent très limités sur le plan sémantique. Par contre, ces outils peuvent être utiles pour contrer le « syndrome de la page blanche » et générer des babillages insignifiants... Cela dit, une personne intelligente peut s'en servir en filtrant et corrigeant les idioties.

Soulignons que la Francophonie a maintenant, <a href="https://cedille.ai/">« Cédille »</a>, un « idiot savant » de qualité équivalente à ceux de langue anglaise. D'après mes tests assez limités, « Cédille » est visiblement meilleur en français que les GPT-X multilingues. Aussi, ses réponses son « culturellement » teintées de la culture francophone. Par exemple, il va faire des allusions au Québec, Canada, France, etc.

Notons en terminant que les avancées récentes en traduction automatique neuronale, les modèles neuronaux du langage et le mécanisme d’attention sur lequel est basé l’architecture Transformer utilisée par BERT et GPT-X originent en grande partie des travaux pionniers du MILA et de l’équipe de Yoshua Bengio de l’UdeM qui ont été perfectionnés et industrialisés par Google et OpenAI.

### Sources d'inspiration:

Discussions toujours passionnantes avec mon ami <a href=https://recherche.umontreal.ca/nos-chercheurs/repertoire-des-professeurs/chercheur/is/in15254/>Patrick Drouin</a>, Ph.D. professeur titulaire en traduction et terminologie à l’Université de Montréal.  

https://fr.wikipedia.org/wiki/Cha%C3%AEne_de_Markov

https://www.nltk.org/

https://www.nltk.org/book/

http://stackoverflow.com/questions/18391602/what-does-generate-do-when-using-nltk-in-python

http://www.mikesboyle.com/post/117202964694/python-nltk-wtf-chapter-1-notes-on-things-that

http://www.gilesthomas.com/2010/05/generating-political-news-using-nltk/

http://www.cs.princeton.edu/courses/archive/spr05/cos126/assignments/markov.html

http://stackoverflow.com/users/55562/lakshman-prasad

http://stackoverflow.com/questions/32441605/generating-ngrams-unigrams-bigrams-etc-from-a-large-corpus-of-txt-files-and-t


## Version avec un germe et corpus fourni avec lien vers un fichier

In [102]:
import random
import nltk
from nltk import wordpunct_tokenize
from nltk.util import ngrams
import re

class ChaineDeMarkov(object):

    def __init__(self, chemin_fichier):
        self.cache = {}
        self.chemin_fichier_corpus = chemin_fichier
        self.lexique = self.lire_corpus()
        self.taille_lexique = len(self.lexique)
        self.cache_de_trigrammes()

    def lire_corpus(self):
        self.fichier_corpus = open(self.chemin_fichier_corpus,'r')
        self.fichier_corpus.seek(0)
        donnees = self.fichier_corpus.read()
        lexique = wordpunct_tokenize(donnees)
        return lexique

    def generer_ngrammes(self, n):
        return list(ngrams(self.lexique, n))

    def cache_de_trigrammes(self):
        for gramme1, gramme2, gramme3 in self.generer_ngrammes(3):
            cle = (gramme1, gramme2)
            if cle in self.cache:
                self.cache[cle].append(gramme3)
            else:
                self.cache[cle] = [gramme3]

    def generation_texte(self, taille=25, germe=None):
        if (not germe) or (len(germe.split(" "))< 2):
            germe = random.randint(0, self.taille_lexique-3)
            gramme1, gramme2 = self.lexique[germe], self.lexique[germe+1]
        else:
            gramme1, gramme2 = germe.split(" ")[-2:]
        mots_generes = []
        for i in range(taille):
            mots_generes.append(gramme1)
            try:
                gramme1, gramme2 = gramme2, random.choice(self.cache[(gramme1, gramme2)])
            except:
                germe = random.randint(0, self.taille_lexique-3)
                gramme1, gramme2 = self.lexique[germe], self.lexique[germe+1]
        mots_generes.append(gramme2)
        texte_genere = ' '.join(mots_generes)
        texte_genere = texte_genere[0:texte_genere.rfind(".")+1]
        texte_genere = re.sub(r"\s([.!?,;:])|\s(['-])\s", r'\g<1>\g<2>', texte_genere)
        return texte_genere[0].upper() + texte_genere[1:]

print("Code chaine de Markov!")

Code chaine de Markov!


## Entraînement d'un modèle Markov trigrammes

### Avec un corpus « Le Coeur a ses raisons » de Marc Labrèche, créé en moissonnant la Toile

##### Sources: 

https://lecoeurasesraisons.fandom.com/fr/wiki/Citations

https://www.narcity.com/fr/montreal/les-20-meilleures-citations-de-criquette-rockwell-dans-le-coeur-a-ses-raisons

In [103]:
casr_chemin_fichier_txt = "DATA/Marc_Labrèche.txt"
markov = ChaineDeMarkov(casr_chemin_fichier_txt)

## Génération d'un texte avec un germe de départ

Note: le paramètre «taille» contrôle la taille du texte en sortie.

Si le paramètre «germe» est vide ou absent, un germe sera choisi aléatoirement

In [104]:
# Génération d'un texte aléatoire
texte_genere = markov.generation_texte(taille=150,germe="Brett:")
texte_genere = texte_genere.replace(" ' ","'").replace(" ’ ","'").replace("*",",").replace(" - ","-").replace('cest','ces').replace('Cest','ces')
print(texte_genere)

, Brett: Cette literie satinée a transformé mon lit en une véritable glissoire de la bouche de Doug, sans le vouloir, contre leur gré, quoi. Brett: Tu sembles tenir la forme? Brad: Brad!! Brad: Oublions nos vieilles querelles... Serrons-nous. Ils s'embrassent. Rideau. Brett: Ne vous inquiètez pas Criquette. Lorsqu'un sourire reptilien. Brett: Oui.. C'est ça?! Brett: Wow. Pardonnez-moi, Brett? Brett: Vous avez fait votre choix? Ridge: Ridge pense... Malgré un doctorat en philosophie et une maîtrise en danse africaine, je?? Doug: Je ne sais pas. Sortons nos jumelles de poches. Si pratiques lors des voyages ou des safaris photos. Ils sortent des jumelles imaginaires.


### Avec un corpus de textes de « Mathieu Bock-Côté » recueillis sur le site du Journal de Montréal

In [105]:
mbc_chemin_fichier_ = "DATA/Mathieu_Bock-Côté.txt"
markov = ChaineDeMarkov(mbc_chemin_fichier_)

In [106]:
# Génération d'un texte avec le germe "Québec libre"
texte_genere = markov.generation_texte(taille=150,germe="Québec libre")
mathieu_bot_cote = texte_genere.replace(" ' ","'").replace(" ’ ","'").replace("*",",").replace(" - ","-").replace('cest','ces').replace('Cest','ces')
print(mathieu_bot_cote)

Québec libre. Quelle sera la véritable raison du fait qu'on en arrive à la veille que cela. Après avoir compilé les études portant sur les enjeux du jour au lendemain de l'histoire politique a-t-il? La CAQ est en train de marquer une claire volonté indépendantiste suffirait-il de la loi sur la route du temps, pour rappeler que le sien sans l'évolution des prénoms américains ( Logan, Tyler, Jayden, etc.), les deux avenirs possibles de nombreux pays. Je vous donne-t-il pas naturellement les animer. Plus l'identité québécoise sans un gouvernement enthousiaste désireux de ne pas arriver ». Le sens du silence, de la droite, étrangement, la loi.


In [116]:
# Génération d'un texte aléatoire - sans germe
texte_genere = markov.generation_texte(taille=150)
texte_genere = texte_genere.replace(" ' ","'").replace(" ’ ","'").replace("*",",").replace(" - ","-").replace('cest','ces').replace('Cest','ces')
print(texte_genere)

: Que gagnerons-nous dérobé de l'éternel a maudite. Lémec, sept personnes. Les frères de Joseph, en présence duquel ont marché avec moi; et voici, la sage-femme dit: Vous me troublez, en sorte que, le froid et la femme de son armée. Dieu se pourvoira lui-même de l'assentiment de Hamor. Le serviteur courut au-dessus de la crème et du vin, Et tu seras dégagé du serment que je ne parlerai plus que cette fois. Adam donna à Rebecca; il donna à ésaü, son père et de là que tu leur conserves la vie. Je t'ai élevé la voix de Jacob qui vinrent avec Jacob en égypte, et partirent.


### Avec un corpus biblique en français  provenant de la bibliothèque NLTK

In [117]:
# Utilisé pour générer le fichier texte_biblique.txt à partir de NLTK

# texte_genese_multilingue = nltk.Text(nltk.corpus.genesis.words())

# # Extraction du corpus NLTK de la version française du texte de la Genèse 
# texte_genese_francais = " ".join(texte_genese_multilingue.tokens[121338:167454])

# # Sauvegarde d'une copie sur di=sque pour assurer la compatibilité avec l'API de la fonction ChaineDeMarkov
# with open("DATA/texte_biblique.txt",'w') as fichier:
#     fichier.write(texte_genese_francais)

In [118]:
markov = ChaineDeMarkov("DATA/texte_biblique.txt")

texte_genere = markov.generation_texte(150, "Dieu dit")
texte_genere = texte_genere.replace(" ' ","'").replace(" ’ ","'").replace("*",",").replace(" - ","-").replace('cest','ces').replace('Cest','ces')

print(texte_genere)

Dieu dit: Que cherches-tu fait? La femme vit que l'un demi-sicle, et se rendit vers Pharaon. Pharaon dit à Laban: Donne-moi. Et ils firent entendre de grandes plaies Pharaon et à ta race. On fit descendre Joseph en égypte, et se présentèrent devant Joseph. Ils s'approchèrent, et il y avait du pain contre vos troupeaux, si l'entrée d'ésaü: elle enfanta un fils. Et Laban lui donna le même ordre au premier-né, que ton âme me bénisse. Isaac dit: Si vous êtes des espions.


In [110]:
print("Fin du carnet web IPython!")

Fin du carnet web IPython!
